# Assignment 2

Before working on this assignment please read these instructions fully. In the submission area, you will notice that you can click the link to **Preview the Grading** for each step of the assignment. This is the criteria that will be used for peer grading. Please familiarize yourself with the criteria before beginning the assignment.

An NOAA dataset has been stored in the file `data/C2A2_data/BinnedCsvs_d400/fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89.csv`. This is the dataset to use for this assignment. Note: The data for this assignment comes from a subset of The National Centers for Environmental Information (NCEI) [Daily Global Historical Climatology Network](https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/readme.txt) (GHCN-Daily). The GHCN-Daily is comprised of daily climate records from thousands of land surface stations across the globe.

Each row in the assignment datafile corresponds to a single observation.

The following variables are provided to you:

* **id** : station identification code
* **date** : date in YYYY-MM-DD format (e.g. 2012-01-24 = January 24, 2012)
* **element** : indicator of element type
    * TMAX : Maximum temperature (tenths of degrees C)
    * TMIN : Minimum temperature (tenths of degrees C)
* **value** : data value for element (tenths of degrees C)

For this assignment, you must:

1. Read the documentation and familiarize yourself with the dataset, then write some python code which returns a line graph of the record high and record low temperatures by day of the year over the period 2005-2014. The area between the record high and record low temperatures for each day should be shaded.
2. Overlay a scatter of the 2015 data for any points (highs and lows) for which the ten year record (2005-2014) record high or record low was broken in 2015.
3. Watch out for leap days (i.e. February 29th), it is reasonable to remove these points from the dataset for the purpose of this visualization.
4. Make the visual nice! Leverage principles from the first module in this course when developing your solution. Consider issues such as legends, labels, and chart junk.

The data you have been given is near **Ann Arbor, Michigan, United States**, and the stations the data comes from are shown on the map below.

In [2]:
import matplotlib.pyplot as plt
import mplleaflet
import pandas as pd

def leaflet_plot_stations(binsize, hashid):

    station_locations = pd.read_csv('data/BinSize_d{}.csv'.format(binsize))
    
    station_location = station_locations[station_locations['hash'] == hashid]

    lons = station_location['LONGITUDE'].tolist()
    lats = station_location['LATITUDE'].tolist()

    fig = plt.figure(figsize=(8,8))

    plt.scatter(lons, lats, c='r', alpha=0.7, s=200)
    
    return mplleaflet.display()

leaflet_plot_stations(400,'fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

C:\Users\mdomi\anaconda3\lib\site-packages\IPython\core\display.py:701: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


In [1]:
%matplotlib widget
import numpy as np

In [3]:
Data = pd.read_csv('data/fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89.csv')
Data.Date = pd.to_datetime(Data.Date)

In [4]:
# Data = Data[(Data.Date.dt.day != 29) & (Data.Date.dt.month != 2)]
Data['FechaAnual'] = Data.apply(lambda df : str(df.Date)[5:7] + '/' + str(df.Date)[8:10], axis=1)
Data = Data[Data['FechaAnual'] != '02/29' ]

Dat15 = Data[(Data.Date.dt.year == 2015)]
Data = Data[(Data.Date.dt.year >= 2005) & (Data.Date.dt.year < 2014)]
maximos = Data[['Data_Value','FechaAnual']].groupby('FechaAnual').max().reset_index()
minimos = Data[['Data_Value','FechaAnual']].groupby('FechaAnual').min().reset_index()

In [5]:
fig = plt.figure()
plt.plot(maximos['FechaAnual'],maximos['Data_Value']/10, c='indianred', alpha = 0.7, label = '2005-2014 Record Highs')
plt.plot(minimos['FechaAnual'],minimos['Data_Value']/10, c='blue', alpha = 0.3, label = '2005-2014 Record Lows')

fecha = []
for i in range(len(maximos['FechaAnual'])):
    if i%90 == 0:
        fecha.append(maximos['FechaAnual'].iloc[i])
plt.xticks(fecha, rotation = 45)
# plt.yscale('symlog')
plt.grid(True,'both','y')

plt.gca().fill_between(minimos['FechaAnual'],maximos['Data_Value']/10,minimos['Data_Value']/10, color='lightslategray', alpha = 0.15)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [6]:
Dat15MX = Dat15.groupby('FechaAnual').max().reset_index()
Dat15Mn = Dat15.groupby('FechaAnual').min().reset_index()

Dat15MX = Dat15MX[Dat15MX['Data_Value'] > maximos['Data_Value']]
Dat15Mn = Dat15Mn[Dat15Mn['Data_Value'] < minimos['Data_Value']]

plt.scatter(Dat15Mn['FechaAnual'], Dat15Mn['Data_Value']/10, c='blue', s=12, marker = 'v', zorder = 4, label = '2015 exceeding low records')
plt.scatter(Dat15MX['FechaAnual'], Dat15MX['Data_Value']/10, c='darkred', s=12, marker = '^', zorder = 4, label = '2015 exceeding high records')

plt.ylim((-60,45))

plt.xlabel('Day of year')
plt.ylabel('Temperature [°C]')

for spine in plt.gca().spines.values():
    spine.set_visible(False)
plt.tick_params(left = False)

In [7]:
plt.title('Comparison of 2005-2014 weather patterns\nwith 2015 data in SE Michigan')

Text(0.5, 1.0, 'Comparison of 2005-2014 weather patterns\nwith 2015 data in SE Michigan')

In [8]:
plt.legend(title = 'Legend', loc = 4, frameon = False, fontsize=8)

In [9]:
plt.text(minimos['FechaAnual'].iloc[5], -58, '*Information from the 29th of February\nduring Leap years has been removed\nin order to achieve an easier analysis', fontsize=5)

Text(01/06, -58, '*Information from the 29th of February\nduring Leap years has been removed\nin order to achieve an easier analysis')

In [10]:
fig.savefig('Assignment2Correction.png', dpi=200)